# Cortical Actin MSE Analysis- Normalized to Cortical Actin Area

In [1]:
# Imports
import cv2
import numpy as np
import os
import pandas as pd 
from itertools import product

In [2]:
# MSE Function

def computeMSE(folder_path, video_path, save_frames=True,
               frames_output="frames_output", masks_output="masks_output"):

    # Create output folders
    os.makedirs(os.path.join(folder_path, frames_output), exist_ok=True) #Frames output = cortical actin
    os.makedirs(os.path.join(folder_path, masks_output), exist_ok=True) #masks_output = cortical actin used for normalization

    cap = cv2.VideoCapture(os.path.join(folder_path, video_path))
    ret, prev_frame = cap.read()
    if not ret:
        print(f"Error reading {video_path}")
        return None

    #Make videos grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    #Initialize vectors to store values
    raw_mse_vals = []
    cortical_mse_vals = []
    normalized_cortical_mse_vals = []
    cortical_area_vals = []

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Threshold to get cell mask- can change threshold value (50 here) to increase or decrease thresholding
        _, cell_mask = cv2.threshold(gray, 50, 1, cv2.THRESH_BINARY)

        # Create cortical mask via erosion
        erosion_size = 10
        kernel = np.ones((erosion_size, erosion_size), np.uint8)
        eroded = cv2.erode(cell_mask, kernel, iterations=1)
        cortical_mask = cell_mask - eroded

        # Calculate Mean Squared Error
        diff_raw = (gray.astype("float") - prev_gray.astype("float")) ** 2
        diff_cortical = diff_raw * cortical_mask

        raw_mse = diff_raw.sum() #Raw values for MSE
        cortical_mse = diff_cortical.sum() #Cortical Actin MSE
        normalized_cortical_mse = diff_cortical.sum() / (cortical_mask.sum()) #Cortical Actin MSE normalized to the area of cortical actin

        raw_mse_vals.append(raw_mse)
        cortical_mse_vals.append(cortical_mse)
        normalized_cortical_mse_vals.append(normalized_cortical_mse)
        cortical_area_vals.append(cortical_mask.sum())

        # Saves images so you can double check thresholding/erosion
        if save_frames and frame_idx % 20 == 0:
            # Raw & cortical images
            cv2.imwrite(os.path.join(folder_path, frames_output, f"{video_path}_frame{frame_idx}_raw.png"), gray)
            cv2.imwrite(os.path.join(folder_path, frames_output, f"{video_path}_frame{frame_idx}_cortical.png"), (cortical_mask*255).astype("uint8"))
            # Whole-cell mask used for area normalization
            cv2.imwrite(os.path.join(folder_path, masks_output, f"{video_path}_frame{frame_idx}_cellmask.png"), (cell_mask*255).astype("uint8"))

        prev_gray = gray
        frame_idx += 1

    cap.release()

    return {
        "Video": video_path,
        "Cortical_Area": np.mean(cortical_area_vals),
        "Raw_MSE": np.mean(raw_mse_vals),
        "Cortical_MSE": np.mean(cortical_mse_vals),
        "Normalized_Cortical_MSE": np.mean(normalized_cortical_mse_vals)
    }

#Saves results
def main(folder_path, output_csv="MSE_results.csv",
         frames_output="frames_output", masks_output="masks_output"):
    results = []
    videos = [f for f in os.listdir(folder_path) if f.endswith(".avi")]
    videos.sort()

    for video in videos:
        res = computeMSE(folder_path, video, save_frames=True,
                         frames_output=frames_output, masks_output=masks_output)
        if res is not None:
            results.append(res)

    df = pd.DataFrame(results)
    df.to_csv(os.path.join(folder_path, output_csv), index=False)
    return df

# Computations

In [3]:
#Quantification for Fast Gels
folder_path = ''
MSE = main(folder_path)

In [4]:
# See Results for Fast Gels
print("Fast- Mean Squared Error (RAW, Cortical, Normalized Cortical")
print(MSE)
print("Mean: " + str(np.mean(MSE)))

Fast- Mean Squared Error (RAW, Cortical, Normalized Cortical
            Video  Cortical_Area     Raw_MSE  Cortical_MSE  \
0  fast_d1_01.avi         2680.0  19010165.0     4548848.8   
1  fast_d1_02.avi         1709.8  15446464.8     3408117.4   
2  fast_d1_03.avi         1103.0   9909554.0     2460972.0   
3  fast_d1_04.avi         2465.8  13146833.6     4078594.0   
4  fast_d1_05.avi         1587.8  17005007.6     3267512.6   
5  fast_d1_06.avi         4043.8  38563589.8     9476472.8   
6  fast_d1_07.avi         1675.6  10668469.8     2413357.8   
7  fast_d1_08.avi         1761.2  10832664.8     2872097.4   
8  fast_d1_09.avi         2559.4  15976851.8     5241602.8   
9  fast_d1_10.avi         3248.0  14916973.6     4980976.6   

   Normalized_Cortical_MSE  
0              1690.977147  
1              1990.501470  
2              2219.496973  
3              1633.788808  
4              2062.214547  
5              2358.856902  
6              1432.108779  
7              1560.0230

/Users/Callie/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Users/Callie/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [5]:
#Quantification for SG Gels
folder_path = ''
MSE = main(folder_path)

In [6]:
# See Results for SG
print("SG- Mean Squared Error (RAW, Cortical, Normalized Cortical")
print(MSE)
print("Mean: " + str(np.mean(MSE)))

SG- Mean Squared Error (RAW, Cortical, Normalized Cortical
                Video  Cortical_Area     Raw_MSE  Cortical_MSE  \
0        sg_d1_01.avi         1366.4  10501129.8     2541304.8   
1  sg_d1_010_5.08.avi         2495.2  15920346.0     4312672.6   
2        sg_d1_02.avi         1326.6   8789948.8     1867928.4   
3        sg_d1_03.avi         1456.6  13314372.4     3134086.8   
4        sg_d1_04.avi         1039.6   7993804.6     1470107.4   
5        sg_d1_05.avi          574.6   6807317.0     1087359.0   
6        sg_d1_06.avi         1116.0   8973606.2     1755768.2   
7        sg_d1_07.avi         1653.8   8030078.6     1729315.0   
8   sg_d1_08_5.08.avi         1394.4   9358540.2     2288640.4   
9   sg_d1_09_5.08.avi         2483.8   9852183.2     2594635.8   

   Normalized_Cortical_MSE  
0              1854.666174  
1              1722.653164  
2              1406.645075  
3              2153.504674  
4              1419.681885  
5              1863.996667  
6          

/Users/Callie/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Users/Callie/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [7]:
#Quantificaiton for Nuclei
folder_path = ''
MSE = main(folder_path)